In [2]:
import pandas as pd
import numpy as np
import os   


Transporte Publico Madrid:

In [24]:
# Leer el archivo de transporte publico madrid
tpm = pd.read_csv(
    "../data/raw/20193_transporte_publico_madrid.csv",
    sep=";",  
    encoding="utf-8-sig"  
)

print("Columnas disponibles:", tpm.columns)

Columnas disponibles: Index(['TIPO DE TRANSPORTE', 'Ciudad', 'Viajeros y tasas', 'Periodo', 'Total'], dtype='object')


In [13]:
# Normalizamos nombre de las tablas:
tpm.columns = (
    tpm.columns
    .str.strip()          # quita espacios al inicio/fin
    .str.lower()          # minúsculas
    .str.replace(" ", "_")  # espacios -> guiones bajos
    .str.replace("ó", "o")  # opcional: quitar acentos
)

print(tpm.columns)

# Filtrar solo Madrid
tpm_madrid = tpm[tpm["ciudad"] == "28079 Madrid"].copy()



Index(['ï»¿tipo_de_transporte', 'ciudad', 'viajeros_y_tasas', 'periodo',
       'total'],
      dtype='object')


In [ ]:
# Revisar datos nulos
print(tpm.isnull().sum())

# Revisar tipo de datos por columna
print(tpm.dtypes)



ï»¿tipo_de_transporte      0
ciudad                     0
viajeros_y_tasas           0
periodo                    0
total                    408
dtype: int64
ï»¿tipo_de_transporte    object
ciudad                   object
viajeros_y_tasas         object
periodo                  object
total                    object
dtype: object


In [15]:
# Convertir columna 'Total' a numérica
tpm_madrid["total"] = pd.to_numeric(tpm_madrid["total"], errors="coerce")

# Transformar 'Periodo' a fecha real (primer día del mes)
tpm_madrid["periodo"] = pd.to_datetime(
    tpm_madrid["periodo"].str.replace("M", ""),
    format="%YM%m",
    errors="coerce"
)

print(tpm_madrid.head())


                      ï»¿tipo_de_transporte        ciudad  \
972  Transporte urbano regular por autobÃºs  28079 Madrid   
973  Transporte urbano regular por autobÃºs  28079 Madrid   
974  Transporte urbano regular por autobÃºs  28079 Madrid   
975  Transporte urbano regular por autobÃºs  28079 Madrid   
976  Transporte urbano regular por autobÃºs  28079 Madrid   

           viajeros_y_tasas periodo   total  
972  Viajeros transportados     NaT  47.559  
973  Viajeros transportados     NaT  43.701  
974  Viajeros transportados     NaT  40.455  
975  Viajeros transportados     NaT  43.739  
976  Viajeros transportados     NaT  41.487  


Accidentes:

In [17]:
# Cargar dataset de accidentalidad
accidentes = pd.read_csv(
    "../data/raw/2025_Accidentalidad.csv", 
    sep=";",             # muchos CSV de datos abiertos usan ;
    encoding="utf-8-sig" # para evitar problemas de tildes
)

# Ver columnas disponibles
print("Columnas:", accidentes.columns)
print("Dimensiones:", accidentes.shape)
print(accidentes.head())

Columnas: Index(['num_expediente', 'fecha', 'hora', 'localizacion', 'numero',
       'cod_distrito', 'distrito', 'tipo_accidente', 'estado_meteorológico',
       'tipo_vehiculo', 'tipo_persona', 'rango_edad', 'sexo', 'cod_lesividad',
       'lesividad', 'coordenada_x_utm', 'coordenada_y_utm', 'positiva_alcohol',
       'positiva_droga'],
      dtype='object')
Dimensiones: (29653, 19)
  num_expediente       fecha     hora  \
0    2025S000056  01/01/2025  0:49:00   
1    2025S000056  01/01/2025  0:49:00   
2    2025S000056  01/01/2025  0:49:00   
3    2025S000057  01/01/2025  2:20:00   
4    2025S000057  01/01/2025  2:20:00   

                                 localizacion numero  cod_distrito  \
0   CALL. LOPEZ DE HOYOS / CALL. ROS DE OLANO    140             5   
1   CALL. LOPEZ DE HOYOS / CALL. ROS DE OLANO    140             5   
2   CALL. LOPEZ DE HOYOS / CALL. ROS DE OLANO    140             5   
3  GTA. RIO ZANCARA / CALL. ARROYO DE POZUELO      1             9   
4  GTA. RIO ZANC

In [18]:
# Normalizamos columnas:
accidentes.columns = (
    accidentes.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("ó", "o")
    .str.replace("í", "i")
    .str.replace("á", "a")
    .str.replace("é", "e")
    .str.replace("ú", "u")
)

# Nulos
print(accidentes.isnull().sum())

# Porcentaje de nulos
print(accidentes.isnull().mean()*100)

# Duplicados
print("Duplicados:", accidentes.duplicated().sum())
accidentes = accidentes.drop_duplicates()

# Convertir fecha a datetime
accidentes["fecha"] = pd.to_datetime(accidentes["fecha"], format="%d/%m/%Y", errors="coerce")

# Convertir hora
accidentes["hora"] = pd.to_datetime(accidentes["hora"], format="%H:%M:%S", errors="coerce").dt.time

num_expediente              0
fecha                       0
hora                        0
localizacion                0
numero                      0
cod_distrito                0
distrito                    0
tipo_accidente              4
estado_meteorologico     3423
tipo_vehiculo             241
tipo_persona                0
rango_edad                  0
sexo                        0
cod_lesividad           13073
lesividad               13073
coordenada_x_utm           18
coordenada_y_utm           18
positiva_alcohol          115
positiva_droga          29575
dtype: int64
num_expediente           0.000000
fecha                    0.000000
hora                     0.000000
localizacion             0.000000
numero                   0.000000
cod_distrito             0.000000
distrito                 0.000000
tipo_accidente           0.013489
estado_meteorologico    11.543520
tipo_vehiculo            0.812734
tipo_persona             0.000000
rango_edad               0.000000
sexo     

In [19]:
# 1. Eliminar filas con nulos críticos (tipo_accidente es clave)
accidentes = accidentes.dropna(subset=["tipo_accidente"])


In [20]:
# 2. Rellenar valores faltantes con categorías claras
accidentes["estado_meteorologico"] = accidentes["estado_meteorologico"].fillna("desconocido")
accidentes["tipo_vehiculo"] = accidentes["tipo_vehiculo"].fillna("sin_especificar")
accidentes["lesividad"] = accidentes["lesividad"].fillna("sin_lesion")
accidentes["cod_lesividad"] = accidentes["cod_lesividad"].fillna(-1)

In [21]:
# 4. Alcohol y droga → convertir a binario (S=1, N=0)
accidentes["positiva_alcohol"] = accidentes["positiva_alcohol"].map({"S": 1, "N": 0})
accidentes["positiva_alcohol"] = accidentes["positiva_alcohol"].fillna(0)  # si hay nulos → asumimos no positivo

In [22]:
# 5. Droga → demasiado nulo, mejor eliminar la columna
accidentes = accidentes.drop(columns=["positiva_droga"])

Unión de Datasets

Empty DataFrame
Columns: [CCAA, Año, Precio_Indice, Tipo, Sector, Salario_Medio]
Index: []
